# 构建聊天机器人
## 概述

我们将介绍一个示例，说明如何设计和实现基于 LLM 的聊天机器人。该聊天机器人将能够进行对话并记住之前的互动。

## 概念

以下是我们将要使用的一些高级组件：
 - Chat Models聊天模型。
 - Prompt Templates提示模板，简化了组合提示的过程，这些提示结合了默认消息、用户输入、聊天历史记录和（可选）其他检索到的上下文。
 - Chat History聊天历史记录，允许聊天机器人“记住”过去的互动，并在回答后续问题时将其考虑在内。


In [1]:
from dotenv import load_dotenv,find_dotenv
import os 

# 获取你的智谱 API Key
在当前文件下创建一个.env文件，将api-key复制进去，如ZHIPUAI_API_KEY = "api-key"

In [2]:
_ = load_dotenv(find_dotenv())

# Using Language Models

首先，让我们学习如何单独使用语言模型。LangChain 支持多种不同的语言模型，您可以互换使用 - 在下面选择您想要使用的模型！

In [3]:
# from langchain_community.chat_models import ChatBaichuan

# model = ChatBaichuan(
#     temperature=0.5
# )

In [4]:
# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(
#     base_url="https://api.baichuan-ai.com/v1",
#     api_key=os.environ["BAICHUAN_API_KEY"],
#     model="Baichuan3-Turbo",
# )

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    base_url="https://open.bigmodel.cn/api/paas/v4",
    api_key=os.environ["ZHIPUAI_API_KEY"],
    model="glm-4",
)

In [6]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="你好，我叫张三。")])

AIMessage(content='你好，张三，很高兴为你服务。有什么可以帮助你的吗？', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 11, 'total_tokens': 27}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6feab847-2f0f-47e9-b963-2a15d211fdb9-0')

让我们首先直接使用该模型。ChatModel 是 LangChain“Runnable”的实例，这意味着它们公开了一个用于与其交互的标准接口。为了简单地调用该模型，我们可以将消息列表传递给 .invoke 方法。

模型本身没有任何状态概念。例如，如果你问一个后续问题：

In [7]:
model.invoke([HumanMessage(content="我的名字是什么")])

AIMessage(content='作为人工智能助手，我没有办法知道您的名字，除非您告诉我。如果您愿意分享，我很乐意知道您的名字。请随意告诉我。', response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 8, 'total_tokens': 38}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a65e8540-d18b-4331-bdba-ea6da4a7b3fc-0')

为了解决这个问题，我们需要将整个对话历史记录传递到模型中。让我们看看这样做会发生什么：

In [8]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="你好！我是张三"),
    AIMessage(content="你好，张三！请问有什么我可以帮到你的吗？"),
    HumanMessage(content="我的名字是什么")
])

AIMessage(content='你的名字是张三。有什么我可以帮助你的吗？', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 29, 'total_tokens': 42}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6ad37f16-787b-4e50-866c-880970b53de3-0')

现在我们可以看到我们得到了很好的回应！  

这是聊天机器人对话式互动能力的基本理念。那么我们如何才能最好地实现这一点呢？

## Message History-消息历史记录

我们可以使用消息历史记录类来包装我们的模型并使其具有状态。这将跟踪模型的输入和输出，并将它们存储在某个数据存储中。未来的交互将加载这些消息并将它们作为输入的一部分传递到链中。让我们看看如何使用它！  

我们可以导入相关类并设置我们的链，该链包装模型并添加此消息历史记录。这里的关键部分是我们作为 get_session_history 传递的函数。此函数应接受 session_id 并返回消息历史记录对象。此 session_id 用于区分单独的对话，应在调用新链时作为配置的一部分传入（我们将展示如何做到这一点）。

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

现在我们需要创建一个配置，每次将其传递给可运行程序。此配置包含不直接属于输入但仍然有用的信息。在本例中，我们希望包含一个 session_id。这应该如下所示：

In [10]:
config = {
    "configurable":{
        "session_id":"abc2"
    }
}

In [11]:
resp = with_message_history.invoke(
    [HumanMessage(content="你好！我是张三")],
    config=config
)
resp.content

'你好，张三！有什么可以帮助你的吗？'

In [12]:
resp = with_message_history.invoke(
    [HumanMessage(content="我的名字是什么？")],
    config=config
)
resp.content

'你的名字是张三。'

太棒了！我们的聊天机器人现在可以记住我们的事情了。如果我们更改配置以引用不同的 session_id，我们可以看到它会重新开始对话。

In [13]:
config = {
    "configurable":{
        "session_id":"abc3"
    }
}

resp = with_message_history.invoke(
    [HumanMessage(content="我的名字是什么？")],
    config=config
)

resp.content

'作为人工智能助手，我没有办法知道您的名字，除非您告诉我。如果您愿意分享，我可以帮助您进行个性化的交流。请随意告诉我您的名字。'

但是，我们总是可以回到原始对话（因为我们将其保存在数据库中）

In [14]:
config = {
    "configurable":{
        "session_id":"abc2"
    }
}

resp = with_message_history.invoke(
    [HumanMessage(content="我的名字是什么？")],
    config=config
)
resp.content

'你之前告诉我你的名字是张三。如果这是你的名字，那么你就是张三。如果需要我帮你处理任何事情，请告诉我。'

这就是我们如何支持聊天机器人与许多用户进行对话！  

现在，我们所做的只是在模型周围添加一个简单的持久层。我们可以通过添加提示模板来开始使其变得更加复杂和个性化。

## Prompt templates-提示模板

提示模板有助于将原始用户信息转换为 LLM 可以使用的格式。在这种情况下，原始用户输入只是一条消息，我们将其传递给 LLM。现在让我们让它更复杂一点。首先，让我们添加一条带有一些自定义指令的系统消息（但仍然将消息作为输入）。接下来，除了消息之外，我们还将添加更多输入。  

首先，让我们添加一条系统消息。为此，我们将创建一个 ChatPromptTemplate。我们将利用 MessagesPlaceholder 传递所有消息。

In [28]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是一位乐于助人的助手。尽你所能回答所有问题。"),
        # ("system","You are a helpful assistant. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

请注意，这会稍微改变输入类型 - 而不是传递消息列表，现在我们传递一个带有消息键的字典，其中包含消息列表。

In [29]:
resp = chain.invoke({
    "messages":[HumanMessage(content="你好！我是王五")]
})

resp.content

'很高兴遇见你！作为一位乐于助人的助手，我会在力所能及的范围内回答你的问题。有什么可以帮助你的吗？'

我们现在可以将其包装在与之前相同的消息历史记录对象中

In [30]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [31]:
config = {
    "configurable":{
        "session_id":"abc5"
    }
}

In [32]:
resp = with_message_history.invoke(
    [HumanMessage(content="你好！我是赵六")],
    config=config
)

resp.content

'你好，赵六！很高兴再次和你交流。如果你有任何问题或需要帮助，请随时告诉我。'

In [33]:
resp = with_message_history.invoke(
    [HumanMessage(content="我的名字是什么？")],
    config=config
)

In [34]:
resp.content

'你之前告诉我，你的名字是赵六。如果有其他问题，我会尽力帮助你解答。赵六，有什么我可以帮你的吗？'

太棒了！现在让我们把提示变得更复杂一点。假设提示模板现在看起来像这样：

In [43]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是一个乐于助人的助手。请用{language}回答所有问题."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

请注意，我们在提示中添加了新的language输入。现在我们可以调用链并传入我们选择的语言。

In [44]:
resp = chain.invoke({
    "messages":[HumanMessage(content="你好！我是张三")],
    "language":"日文",
})

resp.content

'すみません、誤解を招いたようです。私はあなたに質問を日本語でして、日本語で答えていただきたいと思っていました。よろしくお願いします。\n\nはい、了解しました。質問をどうぞお聞きください。日本語で答えますので、心配ありません。'

现在让我们将这个更复杂的链包装到 Message History 类中。这一次，由于输入中有多个键，我们需要指定用于保存聊天记录的正确键。

In [45]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [46]:
config = {
    "configurable":{
        "session_id":"abc6"
    }
}

In [47]:
resp = with_message_history.invoke(
    {"messages":[HumanMessage(content="你好,我是张小三。")],"language":"英语"},
    config=config
)
resp.content

'Hello, I am Zhang Xiaosan. How can I assist you today?'

In [48]:
resp = with_message_history.invoke(
    {"messages":[HumanMessage(content="你好,我是张小三。")],"language":"韩文"},
    config=config
)
resp.content

'안녕하세요, 저는 장 �小三입니다. 어떻게 도와드릴까요?'

# Managing Conversation History - 管理对话历史记录

构建聊天机器人时需要理解的一个重要概念是如何管理对话历史记录。如果不加以管理，消息列表将无限制增长，并可能溢出 LLM 的上下文窗口。因此，添加一个限制传入消息大小的步骤非常重要。  

重要的是，您需要在提示模板之前但在从消息历史记录中加载以前的消息之后执行此操作。  

我们可以通过在提示前面添加一个简单的步骤来适当修改消息键，然后将该新链包装在消息历史记录类中来实现这一点。首先，让我们定义一个将修改传入消息的函数。让我们让它选择最近的 k 条消息。然后我们可以通过在开头添加它来创建一个新链。

In [49]:
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是一个乐于助人的助手。请尽力回答所有问题并用{language}翻译."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

def filter_messages(messages,k=10):
    return messages[-k:]

chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

现在让我们尝试一下！如果我们创建一个超过 10 条消息的列表，我们可以看到它不再记住早期消息中的信息。

In [54]:
messages = [
    HumanMessage(content="你好，我是王小五"),
    AIMessage(content="你好!"),
    HumanMessage(content="我喜欢吃香草冰淇凌"),
    AIMessage(content="你的品味不错"),
    HumanMessage(content="2+2等于多少"),
    AIMessage(content="4"),
    HumanMessage(content="谢谢"),
    AIMessage(content="不客气!"),
    HumanMessage(content="玩得开心吗?"),
    AIMessage(content="是的!"),
]

In [55]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="我叫什么?")],
        "language": "英文",
    }
)
response.content

"As an AI, I don't have the ability to know your name unless you've previously told me. So, I don't know what your name is."

In [56]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="我最爱的冰淇凌口味是哪种？")],
        "language": "英文",
    }
)
response.content

"根据你提供的信息，你并没有直接告诉我你最喜欢的冰淇凌口味是哪种。但是，如果你想根据一般的描述来猜测，可以假设你可能会喜欢以下口味之一：\n\n- 如果你喜欢浓郁的奶香，你可能会喜欢香草冰淇凌。\n- 如果你偏好独特的茶香，你可能会偏爱抹茶冰淇凌。\n- 如果你喜欢酸甜口味，你可能会喜欢酸奶或者草莓冰淇凌。\n\n不过，这只是基于一般偏好的猜测。要确定你最喜欢的冰淇凌口味，我需要你直接告诉我。 :)\n\nIn English:\n\nMy favorite ice cream flavor is which one?\nBased on the information you provided, you haven't directly told me your favorite ice cream flavor. However, if you want to guess based on the general descriptions, you might like one of the following flavors:\n\n- If you enjoy a rich milk flavor, you might like vanilla ice cream.\n- If you prefer a unique tea flavor, you might favor matcha green tea ice cream.\n- If you like a tangy and sweet taste, you might enjoy yogurt or strawberry ice cream.\n\nThese are just guesses based on general preferences. To determine your absolute favorite ice cream flavor, I would need you to tell me directly. :)"

# Streaming - 流媒体

现在我们有了一个功能聊天机器人。但是，聊天机器人应用程序的一个非常重要的用户体验考虑因素是流式传输。LLM 有时可能需要一段时间才能响应，因此为了改善用户体验，大多数应用程序都会在生成每个令牌时将其流式传输回来。这允许用户查看进度。  

其实这样做非常简单！  

所有链都公开一个 .stream 方法，使用消息历史记录的链也不例外。我们可以简单地使用该方法来获取流式响应。

In [58]:
config = {
    "configurable":{
        "session_id":"abc7"
    }
}

for r in with_message_history.stream(
    {
        "messages":[HumanMessage(content="你好，我是赵小六，给我讲个笑话吧。")],
        "language":"英文"
    },
    config=config
):
    print(r.content,end="|")

Of| course|,| I|'d| be| happy| to| help| with| any| questions| or| tasks| you| have|.| If| you|'d| like| a| joke|,| here|'s| a| light|-hearted| one|:

Why| don|'t| scientists| trust| atoms|?

Because| they| make| up| everything|!| 😄

Now|,| how| can| I| assist| you| today|?||